## Class-based CNN with Pytorch

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms

# MNIST dataset
mnist_train = datasets.MNIST(root='MNIST_data/', train=True,
                        # [0, 255] img -> [0.0, 1.0] torch Tensor
                        transform=transforms.ToTensor(), 
                        download=True)

mnist_test = datasets.MNIST(root='MNIST_data/', train=False,
                        transform=transforms.ToTensor(),
                        download=True)

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# DataLoader : helps loading dataset
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!



/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [ ]:
class MyCNN(torch.nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.drop_rate = 0.3

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=self.drop_rate)) # automatically controls drop_rate

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=self.drop_rate))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
            torch.nn.Dropout(p=self.drop_rate))

        fc = torch.nn.Linear(4 * 4 * 128, 625)
        torch.nn.init.xavier_uniform_(fc.weight)
        self.fc1 = torch.nn.Sequential(
            fc,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=self.drop_rate))

        self.fc2 = torch.nn.Linear(625, 10)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, inputs): # automatically called in __call__
        x = self.layer1(inputs)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.view(x.shape[0], -1)  # reshape to make tensor compatible for fc layer
        x = self.fc1(x)
        outputs = self.fc2(x)
        return outputs

# instantiate CNN model
model = MyCNN()

# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss()    # Softmax is internally computed.
optimize_target_parameters = model.parameters()
optimizer = torch.optim.Adam(optimize_target_parameters, lr=learning_rate)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
log_writer = SummaryWriter(log_dir="logs_pytorch")

def train_one_epoch(train_loader, model, criterion, optimizer, cur_epoch, log_writer):
    model.train()
    avg_cost = 0
    total_batch = len(train_loader)

    for i, (batch_xs, batch_ys) in enumerate(train_loader):
        X, Y = batch_xs.cuda(), batch_ys.cuda() # use GPU
        # X = X.view(-1, 28 * 28)  # Don't need anymore.

        optimizer.zero_grad() # zero the parameter gradients

        # forward + backward : compute gradients
        logits = model(X)
        cost = criterion(logits, Y)
        cost.backward()
        
        optimizer.step()  # update parameters using gradients

        avg_cost += cost / total_batch
    print('Epoch:', '%04d' % (cur_epoch + 1), 'cost =', '{:.9f}'.format(avg_cost.item()))
    log_writer.add_scalar('train/loss', avg_cost.item(), cur_epoch)

def test(test_loader, model, criterion, optimizer, cur_epoch, log_writer):
    model.eval()
    correct = 0
    with torch.no_grad():
        for i, (batch_xs, batch_ys) in enumerate(test_loader):
            X, Y = batch_xs.cuda(), batch_ys.cuda() # use GPU
            logits = model(X)

            pred = logits.argmax(dim=1, keepdim=True)
            correct += pred.eq(Y.view_as(pred)).sum().item()
    accuracy = correct / len(mnist_test)
    print('Test Accuracy:', accuracy)
    log_writer.add_scalar('test/accuracy', accuracy, cur_epoch)

# use GPU and train & test
model.cuda()
for cur_epoch in range(training_epochs):
    train_one_epoch(train_loader, model, criterion, optimizer, cur_epoch, log_writer)
    test(test_loader, model, criterion, optimizer, cur_epoch, log_writer)

Epoch: 0001 cost = 0.260168791
Test Accuracy: 0.986
Epoch: 0002 cost = 0.074517943
Test Accuracy: 0.9904
Epoch: 0003 cost = 0.055924688
Test Accuracy: 0.9905
Epoch: 0004 cost = 0.047971945
Test Accuracy: 0.9921
Epoch: 0005 cost = 0.041353516
Test Accuracy: 0.9925
Epoch: 0006 cost = 0.040307127
Test Accuracy: 0.9931
Epoch: 0007 cost = 0.035317019
Test Accuracy: 0.992
Epoch: 0008 cost = 0.031549413
Test Accuracy: 0.9918
Epoch: 0009 cost = 0.029575754
Test Accuracy: 0.9929
Epoch: 0010 cost = 0.027541213
Test Accuracy: 0.9937
Epoch: 0011 cost = 0.025135811
Test Accuracy: 0.994
Epoch: 0012 cost = 0.026054025
Test Accuracy: 0.9948
Epoch: 0013 cost = 0.024377227
Test Accuracy: 0.9936
Epoch: 0014 cost = 0.023026742
Test Accuracy: 0.9934
Epoch: 0015 cost = 0.021668460
Test Accuracy: 0.9932
